# MFR tomography with anisotropic smoothing


On this page, we try to solve the same inversion problem as in the [previous page](./01-mfr-tomography.ipynb), but with anisotropic smoothing.
We will use the same dataset and model, but we will change the derivative matrix according to the gradient of the scalar function.
Therefore, please compare the results of this page with the previous one to see the effect of the anisotropy.

In [ ]:
import pickle
from pathlib import Path
from pprint import pprint

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import AsinhNorm, ListedColormap, Normalize
from matplotlib.ticker import (
    MultipleLocator,
    PercentFormatter,
)
from mpl_toolkits.axes_grid1 import ImageGrid

from cherab.inversion import Mfr
from cherab.inversion.data import get_sample_data
from cherab.inversion.derivative import Derivative

plt.rcParams["figure.dpi"] = 150

# custom Red colormap extracted from "RdBu_r"
cmap = plt.get_cmap("RdBu_r")
CMAP_RED = ListedColormap(cmap(np.linspace(0.5, 1.0, 256)))

# path to the directory to store the MFR results
store_dir = Path().cwd() / "02-mfr"
store_dir.mkdir(exist_ok=True)

## Prepare the data

Load the same data as in the [previous tomography example](./01-mfr-tomography.ipynb).

In [ ]:
# Load the sample data
grid_data = get_sample_data("bolo.npz")

# Extract the data
grids = grid_data["grid_centres"]
voxel_map = grid_data["voxel_map"].squeeze()
mask = grid_data["mask"].squeeze()
T = grid_data["sensitivity_matrix"]

print(f"{grids.shape = }")
print(f"{voxel_map.shape = }")
print(f"{mask.shape = }")
print(f"{T.shape = }")
print(f"T density: {np.count_nonzero(T) / T.size:.2%}")

### Define Phantom emission profile

The emission profile remains unchanged.

In [ ]:
PLASMA_AXIS = np.array([1.5, 1.5])
LCFS_RADIUS = 1
RING_RADIUS = 0.5

RADIATION_PEAK = 1
CENTRE_PEAK_WIDTH = 0.05
RING_WIDTH = 0.025


def emission_function_2d(rz_point: np.ndarray) -> float:
    direction = rz_point - PLASMA_AXIS
    bearing = np.arctan2(direction[1], direction[0])

    # calculate radius of coordinate from magnetic axis
    radius_from_axis = np.hypot(*direction)
    closest_ring_point = PLASMA_AXIS + (0.5 * direction / radius_from_axis)
    radius_from_ring = np.hypot(*(rz_point - closest_ring_point))

    # evaluate pedestal -> core function
    if radius_from_axis <= LCFS_RADIUS:
        central_radiatior = RADIATION_PEAK * np.exp(-(radius_from_axis**2) / CENTRE_PEAK_WIDTH)

        ring_radiator = (
            RADIATION_PEAK * np.cos(bearing) * np.exp(-(radius_from_ring**2) / RING_WIDTH)
        )
        ring_radiator = max(0, ring_radiator)

        return central_radiatior + ring_radiator
    else:
        return 0


# Create a 2-D phantom
nr, nz = voxel_map.shape
phantom_2d = np.full((nr, nz), np.nan)
for ir, iz in np.ndindex(nr, nz):
    if not mask[ir, iz]:
        continue
    phantom_2d[ir, iz] = emission_function_2d(grids[ir, iz])

# Create a 1-D phantom for the inversion
phantom = phantom_2d[mask]

In [ ]:
fig, ax = plt.subplots(layout="constrained")
image = ax.pcolormesh(
    grids[:, 0, 0],
    grids[0, :, 1],
    phantom_2d.T,
    cmap=CMAP_RED,
)
ax.set_aspect("equal")
cbar = plt.colorbar(image, pad=0.0)
cbar.set_label("[W/m$^3$]")
ax.set_title("Phantom emissivity")
ax.set_xlabel("$R$ [m]")
ax.set_ylabel("$Z$ [m]")
ax.tick_params(axis="both", which="both", direction="in", top=True, right=True)

### Compute the measurement data

Let us prepare the measurement data $\mathbf{b}$ using the geometry matrix $\mathbf{T}$: $\mathbf{b} = \mathbf{T} \mathbf{x} + \mathbf{e}$, where $\mathbf{e}$ represents the noise.
We will introduce 10% Gaussian noise based on the maximum value of the measurement data.

In [ ]:
T /= 4.0 * np.pi  # Divide by 4π steradians for use with power measurements in [W]
data = T @ phantom

rng = np.random.default_rng()
data_w_noise = data + rng.normal(0, data.max() * 1.0e-2, data.size)
data_w_noise = np.clip(data_w_noise, 0, None)

## Anisotropic smoothing matrix $\mathbf{H}$

### Definition

To apply the anisotropic smoothing, we propose the following anisotropic smoothing matrix (regularization matrix) $\mathbf{H}$ <cite data-footcite="Odstrcil2012-ta">(Odstrcil et al. 2012)</cite> :

$$
\mathbf{H}
\equiv
    \mathrm{sig}(\alpha)\mathbf{D}_\parallel^\mathsf{T}\mathbf{W}\mathbf{D}_\parallel
    + \mathrm{sig}(-\alpha)\mathbf{D}_\perp^\mathsf{T}\mathbf{W}\mathbf{D}_\perp,
$$

where $\mathbf{D}_\parallel$ and $\mathbf{D}_\perp$ are derivative matrices in the parallel directions along the gradient of a specific scalar function, $\nabla f(R, Z)$, and perpendicular to it, respectively. The degree of anisotropy is determined by the coefficient $\mathrm{sig}(\alpha)$, which is the sigmoid function with a parameter $\alpha$ that regulates the strength of the anisotropy.

The differentiation of $\mathbf{D}_\parallel$ and $\mathbf{D}_\perp$ depends on the scalar function $f(R, Z)$. We utilize a simple monotonically increasing function from the point $(R_0, Z_0)$, defined as follows:

$$
f(R, Z) = \sqrt{(R - R_0)^2 + (Z - Z_0)^2}.
$$

The gradient and its orthogonal vector are shown below.

In [ ]:
def scalar_func(r, z):
    """Scalar function to be used for the demonstration."""
    return np.hypot(r - PLASMA_AXIS[0], z - PLASMA_AXIS[1])


# Compute scalar function values at each grid point.
fvals = np.zeros_like(grids[:, :, 0])
for ir, iz in np.ndindex(grids.shape[:2]):
    fvals[ir, iz] = scalar_func(grids[ir, iz, 0], grids[ir, iz, 1])

# Compute gradients of scalar function
grad_r, grad_z = np.gradient(fvals, grids[:, 0, 0], grids[0, :, 1])

# Mask values outside the plasma
fvals[voxel_map < 0] = np.nan
grad_r[voxel_map < 0] = np.nan
grad_z[voxel_map < 0] = np.nan

# Let us show the scalar function, its gradient, and orthogonal vectors.
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, layout="constrained")
ax1.pcolormesh(
    grids[:, 0, 0],
    grids[0, :, 1],
    fvals.T,
    cmap=CMAP_RED,
)
ax1.set_aspect("equal")
ax1.set_title("Scalar function $f(R, Z)$")
ax1.set_xlabel("$R$ [m]")
ax1.set_ylabel("$Z$ [m]")

ax2.quiver(
    grids[:, 0, 0][1::3],
    grids[0, :, 1][::3],
    grad_r[1::3, ::3].T,
    grad_z[1::3, ::3].T,
    scale=15,
    color="black",
    width=0.007,
)
ax2.set_aspect("equal")
ax2.set(xlim=ax1.get_xlim(), ylim=ax1.get_ylim())
ax2.set_title("$\\nabla f(R,Z)$")
ax2.set_xlabel("$R$ [m]")

ax3.quiver(
    grids[:, 0, 0][1::3],
    grids[0, :, 1][::3],
    -grad_z[1::3, ::3].T,
    grad_r[1::3, ::3].T,
    scale=15,
    color="black",
    width=0.007,
)
ax3.set_aspect("equal")
ax3.set(xlim=ax1.get_xlim(), ylim=ax1.get_ylim())
ax3.set_title("Orthogonal to $\\nabla f(R,Z)$")
ax3.set_xlabel("$R$ [m]");

In [ ]:
deriv = Derivative(grids, voxel_map)
dmat_para, dmat_perp = deriv.matrix_gradient(scalar_func)

dmat_pairs = [(dmat_para, dmat_para), (dmat_perp, dmat_perp)]
pprint(dmat_pairs)

### Optimize the anisotropic parameter $\alpha$

When reconstructing the profile, we need to choose the optimal value of the anisotropic parameter $\alpha$, which is often optimized by reconstructing several test cases and seeking the best solution in terms of some criteria (e.g. [relative error](https://en.wikipedia.org/wiki/Approximation_error), [SSIM](https://en.wikipedia.org/wiki/Structural_similarity_index_measure), [RMSD](https://en.wikipedia.org/wiki/Root_mean_square_deviation), etc.).

Because the phantom profile in this example is spread out in the circumferential direction to some extent, we try to strengthen the anisotropy in the same direction: set the anisotropic parameter $\alpha$ to be negative.
Let us plot the sigmoid function with setting parameters $\alpha$.

In [ ]:
def sigmoid(x):
    """Sigmoid function."""
    return 1 / (1 + np.exp(-x))

In [ ]:
x = np.linspace(-8, 8)
alphas = np.linspace(-5, 0, 20)  # selected alphas

plt.plot(x, sigmoid(x), label="Sigmoid function", zorder=0)
plt.scatter(alphas, sigmoid(alphas), marker="x", color="red", label="Selected $\\alpha$")
plt.axhline(0.5, color="black", linestyle="--", zorder=-1)
plt.axvline(0, color="black", linestyle="--", zorder=-1)
plt.xlabel("Anisotropic Parameter $\\alpha$")
plt.ylabel("$\\mathrm{sig}(\\alpha)$")
plt.xlim(x.min(), x.max())
plt.legend();

Let us define criterion functions: [relative error](https://en.wikipedia.org/wiki/Approximation_error) $\varepsilon_\mathrm{rel}$ and [structual similarity index](https://en.wikipedia.org/wiki/Structural_similarity_index_measure) $\mathrm{SSIM}$.
They are given as follows:

$$
\begin{align}
\varepsilon_\mathrm{rel} &\equiv \frac{\|\mathbf{x}_\mathrm{true} - \mathbf{x}_\mathrm{rec}\|}{\|\mathbf{x}_\mathrm{true}\|},\\
\mathrm{SSIM} & \equiv \frac{(2\mu_x\mu_y + C_1)(2\sigma_{xy} + C_2)}{(\mu_x^2 + \mu_y^2 + C_1)(\sigma_x^2 + \sigma_y^2 + C_2)},
\end{align}
$$

where $\mathbf{x}_\mathrm{true}$ is the true profile, $\mathbf{x}_\mathrm{rec}$ is the reconstructed profile, $\mu_x, \mu_y$ are the average values of $\mathbf{x}_\mathrm{true}$ and $\mathbf{x}_\mathrm{rec}$, respectively, $\sigma_x, \sigma_y$ are the standard deviations of $\mathbf{x}_\mathrm{true}$ and $\mathbf{x}_\mathrm{rec}$, respectively, $\sigma_{xy}$ is the covariance of $\mathbf{x}_\mathrm{true}$ and $\mathbf{x}_\mathrm{rec}$, $C_1, C_2$ are constants, and $\|\cdot\|$ is the Euclidean norm.
Here $C_1 = (k_1L)^2, C_2 = (k_2L)^2$ are constants, where $k_1=0.01, k_2=0.03$, and $L$ is the dynamic range of the profile values.

In [ ]:
def relative_error(x, true):
    """Compute relative error."""
    return np.linalg.norm(true - x) / np.linalg.norm(true)


def ssim(x, y):
    """Compute Structural Similarity Index."""
    ux, uy = np.mean(x), np.mean(y)
    vx, vy = np.var(x), np.var(y)
    vxy = (x - ux) @ (y - uy) / (x.size - 1)
    x_range, y_range = x.max() - x.min(), y.max() - y.min()
    data_range = max(x_range, y_range)
    c1, c2 = (0.01 * data_range) ** 2, (0.03 * data_range) ** 2
    return (2 * ux * uy + c1) * (2 * vxy + c2) / ((ux**2 + uy**2 + c1) * (vx + vy + c2))

Let us try to solve the problem with the MFR method varying the anisotropic parameter $\alpha$.

In [ ]:
num_mfi = 4  # number of MFR iterations
eps = 1.0e-6  # small positive number to avoid division by zero
tol = 1.0e-2  # tolerance for the convergence criterion

mfr = Mfr(T, dmat_pairs, data=data_w_noise)

errors = []
ssims = []

for alpha in alphas:
    sol, stats = mfr.solve(
        derivative_weights=[sigmoid(alpha), sigmoid(-alpha)],
        miter=num_mfi,
        eps=eps,
        tol=tol,
        store_regularizers=False,
        spinner=False,
    )
    errors.append(relative_error(sol, phantom))
    ssims.append(ssim(phantom, sol))

Plot the relative error and SSIM for each $\alpha$. Lower relative error and higher SSIM are better.

In [ ]:
fig, ax1 = plt.subplots(layout="constrained")
ax2 = ax1.twinx()

# Plot the relative error
ax1.plot(alphas, errors, marker=".")
ax1.set_xlabel("Anisotropic Parameter $\\alpha$")
ax1.set_ylabel("Relative Error")
ax1.set_title("Effect of Anisotropic Regularization")
ax1.yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax2.spines["left"].set_color("C0")
ax1.yaxis.label.set_color("C0")
ax1.tick_params(axis="y", colors="C0")
line1 = ax1.axvline(
    alphas[np.argmin(errors)], color="C0", linestyle="--", lw=1, label="Minimum Error"
)


# Plot the SSIM
ax2.plot(alphas, ssims, marker=".", color="C1")
ax2.set_ylabel("SSIM")
ax2.yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax2.spines["right"].set_color("C1")
ax2.yaxis.label.set_color("C1")
ax2.tick_params(axis="y", colors="C1")
line2 = ax2.axvline(
    alphas[np.argmax(ssims)], color="C1", linestyle="--", lw=0.7, label="Maximum SSIM"
)

lines = [line1, line2]
ax2.legend(lines, [line.get_label() for line in lines], loc=0);

## Evaluate the best reconstruction with the optimized $\alpha$

We are assessing the optimal reconstruction using the optimized parameter $\alpha$, which aims to maximize the SSIM.

In [ ]:
opt_alpha = alphas[np.argmax(ssims)]

sol, stats = mfr.solve(
    derivative_weights=[sigmoid(opt_alpha), sigmoid(-opt_alpha)],
    miter=num_mfi,
    eps=eps,
    tol=tol,
    store_regularizers=True,
    spinner=False,
    path=store_dir,
)

Let's plot the L-curve and its curvature to use as a regularization criterion, so we can determine the optimal value of the regularization parameter.

In [ ]:
lcurve = stats["regularizer"]
fig, axes = plt.subplots(1, 2, dpi=150, figsize=(9, 4), layout="constrained")
lcurve.plot_L_curve(fig, axes[0])
lcurve.plot_curvature(fig, axes[1]);

### Compare profiles

Now, we will compare the reconstructed profile with the phantom and assess the quality of the reconstruction.

In [ ]:
vmax = max(np.abs(phantom).max(), np.abs(sol).max())

fig = plt.figure(figsize=(10, 5))
axes = ImageGrid(fig, 111, nrows_ncols=(1, 2), axes_pad=0.05, cbar_mode="single")
norm = Normalize(vmin=-vmax, vmax=vmax)
for ax, profile, label in zip(
    axes.axes_all,
    [phantom, sol],
    ["Phantom", f"Reconstruction ($\\alpha_{{\\mathrm{{opt}}}} = ${opt_alpha:.2f})"],
    strict=True,
):
    profile_2d = np.full(voxel_map.shape, np.nan)
    profile_2d[mask] = profile

    ax.pcolormesh(
        grids[:, 0, 0],
        grids[0, :, 1],
        profile_2d.T,
        cmap="RdBu_r",
        norm=norm,
    )
    ax.set_title(label)
    ax.set_xlabel("$R$ [m]")

    ax.tick_params(axis="both", which="both", direction="in", top=True, right=True)

axes[0].set_ylabel("$Z$ [m]")
mappable = ScalarMappable(cmap="RdBu_r", norm=norm)
cbar = plt.colorbar(mappable, cax=axes.cbar_axes[0])
cbar.set_label("Emissivity [W/m$^3$]");

In [ ]:
dr, dz = grids[1, 0, 0] - grids[0, 0, 0], grids[0, 1, 1] - grids[0, 0, 1]
rmin, rmax = grids[0, 0, 0] - dr * 0.5, grids[-1, 0, 0] + dr * 0.5
zmin, zmax = grids[0, 0, 1] - dz * 0.5, grids[0, -1, 1] + dz * 0.5

vmax = max(np.amax(np.abs(sol)), np.amax(np.abs(phantom)))
levels = np.linspace(-vmax, vmax, 20)
sol_2d = np.full_like(phantom_2d, np.nan)
sol_2d[mask] = sol
norm = Normalize(vmin=-vmax, vmax=vmax)

fig, ax = plt.subplots(layout="constrained")
for profile, ls in zip([phantom_2d, sol_2d], ["--", "-"], strict=True):
    ax.contour(
        profile.T,
        cmap="RdBu_r",
        norm=norm,
        linestyles=ls,
        levels=levels,
        extent=[rmin, rmax, zmin, zmax],
    )

proxy = [plt.Line2D([], [], c="k", ls="dotted"), plt.Line2D([], [], c="k", ls="solid")]
ax.legend(proxy, ["Phantom", "Reconstruction"], loc="upper right")

ax.set_aspect("equal")
ax.set_xlabel("$R$ [m]")
ax.set_ylabel("$Z$ [m]")
ax.set_title("Contour plot of Phantom and Reconstruction")
ax.tick_params(axis="both", which="both", direction="in", top=True, right=True)

mappable = ScalarMappable(cmap="RdBu_r", norm=norm)
cbar = plt.colorbar(mappable=mappable, ax=ax, pad=0.0)
cbar.set_label("W/m$^3$")

### Compare the measurement powers

The measured power calculated by multiplying the geometry matrix by the emission vector and the back-calculated power calculated by multiplying the geometry matrix by the inverted emissivity are all in good agreement.

In [ ]:
back_calculated_measurements = T @ sol

fig, (ax1, ax2) = plt.subplots(2, 1, layout="constrained", figsize=(8, 6), sharex=True)

# Plot the phantom and the back-calculated measurements
ax1.bar(np.arange(data.size), data, label="Matrix-based measurements")
ax1.plot(back_calculated_measurements, ".", color="C1", label="Back-calculated from inversion")
ax1.legend()
ax1.set_ylabel("Power [W]")
ax1.ticklabel_format(style="sci", axis="y", useMathText=True)
ax1.tick_params(axis="both", which="both", direction="in", top=True, right=True)
ax1.set_xlim(-1, data.size)
ax1.xaxis.set_major_locator(MultipleLocator(base=5))
ax1.xaxis.set_minor_locator(MultipleLocator(base=1))

# Plot the residuals between the measurements b and the back-calculated Tx (b - Tx)
ax2.axhline(0, color="k", linestyle="--")
ax2.bar(np.arange(data.size), data - back_calculated_measurements, color="C2")
ax2.set_xlabel("channel of bolometers")
ax2.set_ylabel("Residual [W]")
ax2.ticklabel_format(style="sci", axis="y", useMathText=True)
ax2.tick_params(axis="both", which="both", direction="in", top=True, right=True)

In [ ]:
print(
    "The relative error of power measurements is",
    f"{relative_error(back_calculated_measurements, data):.2%}",
)

### Quantitative evaluation

We evaluate the reconstruction quality by measuring quantitative metrics such as relative error, SSIM, and the total/negative power of the reconstructed profile.

In [ ]:
dr = grids[1, 0, 0] - grids[0, 0, 0]
dz = grids[0, 1, 1] - grids[0, 0, 1]

volumes = dr * dz * 2.0 * np.pi * grids[..., 0]
volumes = volumes[mask]

print(f"Relative error : {relative_error(sol, phantom):.2%}")
print(f"SSIM           : {ssim(sol, phantom):.2%}")
print("--------------------------------------------")
print(f"Total power of phantom       : {phantom @ volumes:.4g} W")
print(f"Total power of reconstruction: {sol @ volumes:.4g} W")
print(f"Total negative power of reconstruction: {sol[sol < 0] @ volumes[sol < 0]:.4g} W")

### Solution basis

The [solution bases](../../user/theory/inversion.ipynb#Series-expansion-of-the-solution) from 0-th to 18-th are shown below.

In [ ]:
# Plot solution bases
fig = plt.figure(figsize=(16, 10))
axes = ImageGrid(
    fig,
    111,
    nrows_ncols=(3, 6),
    axes_pad=0.0,
    cbar_mode=None,
)

for i, ax in enumerate(axes.axes_all):
    profile2d = np.full(voxel_map.shape, np.nan)
    profile2d[mask] = lcurve.basis[:, i]

    absolute = np.abs(lcurve.basis[:, i]).max()
    norm = AsinhNorm(vmin=-1 * absolute, vmax=absolute, linear_width=absolute * 1e-1)

    ax.pcolormesh(
        grids[:, 0, 0],
        grids[0, :, 1],
        profile2d.T,
        cmap="RdBu_r",
        norm=norm,
    )
    # ax.set_aspect("equal")
    ax.set_xlabel("$R$ [m]") if i >= 12 else None
    ax.set_ylabel("$Z$ [m]") if i % 6 == 0 else None
    ax.tick_params(axis="both", which="both", direction="in", top=True, right=True)

    ax.text(
        0.5,
        0.93,
        f"basis {i}",
        horizontalalignment="center",
        verticalalignment="center",
        transform=ax.transAxes,
    )

## Iteration history


As described in the [MFR definition section](../../user/theory/mfr.ipynb), MFR is the iterative method.
To see the convergence behavior, we investigate the iteration history.

### Reconstruction profiles history

Firstly, let's examine the solution for each iteration.
To address negative values, we will plot solutions using the arcsinh scale.

In [ ]:
# Stored regularizer files
reg_files = list(store_dir.glob("*.pickle"))
reg_files = sorted(reg_files, key=lambda x: int(x.stem.split("_")[-1]))

# Load each solution
sols = []
regs = []
for file in reg_files:
    with open(file, "rb") as f:
        reg = pickle.load(f)

    sols.append(reg.solution(reg.lambda_opt))
    regs.append(reg)

profiles = [phantom] + sols

# set vmin and vmax for all solutions
vmin = min(profile.min() for profile in profiles)
vmax = max(profile.max() for profile in profiles)


# Plot the solutions
fig = plt.figure(figsize=(10, 5))
axes = ImageGrid(
    fig,
    111,
    nrows_ncols=(1, len(profiles)),
    axes_pad=0.05,
    cbar_mode="single",
    cbar_location="right",
)

absolute = max(abs(vmax), abs(vmin))
linear_width = absolute * 0.1  # quasi-linear region width
norm = AsinhNorm(linear_width=linear_width, vmin=-1 * absolute, vmax=absolute)
i = 0
for ax, profile in zip(axes.axes_all, profiles, strict=True):
    sol_2d = np.full(voxel_map.shape, np.nan)
    sol_2d[mask] = profile

    ax.pcolormesh(
        grids[:, 0, 0],
        grids[0, :, 1],
        sol_2d.T,
        cmap="RdBu_r",
        norm=norm,
    )
    if i == 0:
        ax.set_title("Phantom")
    else:
        ax.set_title(f"MFR iteration {i}")

    ax.set_xlabel("$R$ [m]")
    i += 1

    ax.tick_params(axis="both", which="both", direction="in", top=True, right=True)

axes[0].set_ylabel("$Z$ [m]")
mappable = ScalarMappable(norm=norm, cmap="RdBu_r")
cbar = plt.colorbar(mappable, cax=axes.cbar_axes[0])
cbar.set_label("Emissivity [W/m$^3$]")

### Quantitative evaluation history

The following plots show the quantitative evaluation changes during the iteration.

In [ ]:
relative_errors = []
total_powers = []
negative_powers = []
ssims = []

for sol in sols:
    relative_errors.append(relative_error(sol, phantom))
    total_powers.append(sol @ volumes)
    negative_powers.append(sol[sol < 0] @ volumes[sol < 0])
    ssims.append(ssim(phantom, sol))

# Append nan value to the last iteration
relative_errors.append(np.nan)
negative_powers.append(np.nan)
ssims.append(np.nan)

# show each values as a bar plot
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, layout="constrained", sharex=True, figsize=(4, 6))

x = np.arange(1, len(relative_errors) + 1)  # the label locations
rects = ax1.bar(x[:4], total_powers, color="C0", label="Reconstruction")
ax1.bar_label(rects, padding=-15, fmt="{:.3f}", color="w")
rects = ax1.bar(x[-1], np.sum(phantom * volumes), color="C1", label="Phantom")
ax1.bar_label(rects, padding=-15, fmt="{:.3f}", color="w")
ax1.set_ylabel("Total power [W]")
ax1.tick_params(axis="both", which="both", direction="in", top=True, right=True)
ax1.ticklabel_format(style="sci", axis="y", useMathText=True)


rects = ax2.bar(x, negative_powers, color="C2")
ax2.bar_label(rects, padding=3, fmt="{:.3f}")
ax2.set_ylim(ymin=np.nanmin(negative_powers) * 1.3)
ax2.set_ylabel("Negative power [W]")
ax2.tick_params(axis="both", which="both", direction="in", top=True, right=True)
ax2.ticklabel_format(style="sci", axis="y", useMathText=True)

rects = ax3.bar(x, relative_errors, color="C3")
ax3.bar_label(rects, padding=3, fmt="{:.1%}")
ax3.set_ylim(ymax=np.nanmax(relative_errors) * 1.3)
ax3.set_ylabel("Relative error")
ax3.tick_params(axis="both", which="both", direction="in", top=True, right=True)
ax3.yaxis.set_major_formatter(PercentFormatter(xmax=1))

rects = ax4.bar(x, ssims, color="C4")
ax4.bar_label(rects, padding=3, fmt="{:.1%}")
ax4.set_ylim(ymax=np.nanmax(ssims) * 1.3)
ax4.set_ylabel("SSIM")
ax4.set_xlabel("MFR iteration")
ax4.set_xticks(x)
ax4.set_xticklabels(x.tolist()[:4] + ["Phantom"])
ax4.tick_params(axis="both", which="both", direction="in", top=True, right=True)
ax4.yaxis.set_major_formatter(PercentFormatter(xmax=1))

### Solution basis history

Solution bases are altered and localized to specific regions during the iteration process.


In [ ]:
# Plot solution bases
fig = plt.figure(figsize=(16, 10))

for j, reg in enumerate(regs):
    axes = ImageGrid(
        fig,
        (len(sols), 1, j + 1),
        nrows_ncols=(1, 6),
        axes_pad=0.0,
        cbar_mode=None,
    )

    axes[0].set_ylabel(f"Iteration {j + 1}")

    for i, ax in enumerate(axes.axes_all):
        profile2d = np.full(voxel_map.shape, np.nan)
        profile2d[mask] = reg.basis[:, i]

        absolute = max(abs(reg.basis[:, i].min()), abs(reg.basis[:, i].max()))
        norm = AsinhNorm(vmin=-1 * absolute, vmax=absolute, linear_width=absolute * 1e-1)

        ax.pcolormesh(
            grids[:, 0, 0],
            grids[0, :, 1],
            profile2d.T,
            cmap="RdBu_r",
            norm=norm,
        )
        ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

        ax.text(
            0.5,
            0.93,
            f"basis {i}",
            horizontalalignment="center",
            verticalalignment="center",
            transform=ax.transAxes,
        )

fig.subplots_adjust(hspace=0.1)

## References
